In [1]:
import pandas as pd
import numpy as np

c:\Users\giuli\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\giuli\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\giuli\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


# Term structure

$$
B^{*}(t, T_x)=e^{-h^{*}(t, T) \cdot T_x} \quad ; T_x \in[0,T]
$$

$$
B(t, T)=[1+i(t, T)]^{-(T-t)}
$$

In [2]:
def annual_spot_prices(x,yields=True):
    """
    Compute annual spot prices from the yield structure
    
    Args:
        yields (np array): Yield structure, annual frequency
    
    Returns:
        prices_ (np array): Annual bond prices
    
    """
    t=np.linspace(0,len(x)-1,len(x))+1
    if yields==True:
        prices_=np.exp(np.negative(x)*t)
    else:
        prices_=(1+x)**(-t)
    return prices_

--------------------------


* Change frequency of the spot prices assuming an intra-year linearity:



In [3]:
def interpolate(x,freq):
    """
    Interpolate between two points, given a certain freqency
    
    Args:
        x (np array): 1-D array with two values i.e. [1, 1.5]
        freq (int): number of intervals in one year
    
    Returns:
        m (np array): Interpolated points between
    
    """
    freq+=2
    alpha=np.linspace(0,1,freq-1)
    m=x[1]*(alpha)+x[0]*(1-alpha)
    return m

In [4]:
def linear_interp(X,freq):
    """
    Interpolate the bond price structure, assuming an intra-year linearity
    
    Args:
        x (np array): 1-D array of the original bond price structure
        freq (int): number of intervals in one year
    
    Returns:
        l (np array): Interpolated price structure
    
    """
    l=[]
    for i in range(0,len(X)-1):
        l.append(interpolate((X[i:i+2]),freq=freq)[:-1])
    l.append(interpolate((X[i:i+2]),freq=freq)[-1:])
    l=np.concatenate(l)

    return l

Forward spot prices:
$$
B^{*}(t, s, T)=\frac{B^{*}(t, T)}{B^{*}(t, s)}
$$

In [5]:
def forward_prices(x):
    """
    Compute forward spot prices
    
    Args:
        x (np array): 1-D array of the original bond price structure
    
    Returns:
        x (np array): 1-D array of the forwar prices
    
    """
    y=x[1:]/x[:-1]
    y=np.concatenate([[x[0]],y])
    return y

$$
i(0, t-1, t)=\frac{(1+i(0, t))^{t}}{(1+i(0, t-1))^{t-1}}-1
$$

In [6]:
def forward_rates(x):
    """
    Compute forward rates, given rates structure
    
    Args:
        x (np array): 1-D array of the original rates structure
    
    Returns:
        x (np array): 1-D array of the forward rates
    
    """
    y=np.array(x[0])
    t=np.linspace(1,len(x)-1,len(x)-1)+1
    z=(1+x[1:])**t/((1+x[:-1])**(t-1))-1
    y=np.concatenate([[x[0]],z])
    return y

Interest from price:
$$
i^{*}(t, s, T)=B^{*}(t, s, T)^{-\frac{1}{T-s}}-1
$$

In [7]:
def interest_from_price(x,freq=1):
    t=np.linspace(0,len(x)-1,len(x))+1
    y=x**(-freq*(1/t))-1
    #y=np.concatenate([[1],y])
    return y

# Products

**FLOATER**: it's a coupon bond whose first coupon $c_{1}$ is known and the others are related to a term structure of interest $s_{i}$; it can als pay a spread $s$.

* Classic formula:
$$
F l=c_{1} B\left(0, t_{1}\right)+\sum_{i=2}^{n} c_{i} B\left(0, t_{i}\right)+s \sum_{i=2}^{n} B\left(0, t_{i}\right)
$$

* Multi period formula:
$$
Floater= \frac{coupon}{frequency} \sum_{p=1}^{freq} B(0,\frac{p}{freq}) + \sum_{s=2}^{T} \sum_{p=1}^{freq}  \{ i(0,s-1 + \frac{p-1}{freq},s-1+\frac{p}{freq})*B(0,s-1+\frac{p}{freq})\} + B(0,T)
$$

Reverse Floater: we have a floater whose coupon are computed as a maximum coupon $\bar{c}$ reduced by a certain quantity related to the index: $c_{i}=\bar{c}-\alpha i(0, s, t)$.

**CONVENTION**:
* MARKET CONVENTION: the discount curve $B(0,t)$ is considered the same as the market 
* WITHOUT MARKET CONVENTION: the discount curve is different from the product term structure (yields from product, bonds from market)


**Defaultable coupon bond** formula:
$$
D P_{0}=c_{6} \sum_{s=0}^{2} \sum_{k=1}^{6} B^{*}\left(0, s+\frac{k}{6}\right)+c_{12} \sum_{s=2}^{3} \sum_{k=1}^{12} B^{*}\left(0, s+\frac{k}{12}\right)+C_{n}^{\prime} B^{*}(0,3)=48.82
$$

In [8]:
st=[0.998501124,
0.997004496,
0.993719803,
0.991635181,
0.988565872,
]

In [9]:
linear_interp(np.array(st), freq=24)

array([0.99850112, 0.99843876, 0.9983764 , 0.99831405, 0.99825169,
       0.99818933, 0.99812697, 0.99806461, 0.99800225, 0.99793989,
       0.99787753, 0.99781517, 0.99775281, 0.99769045, 0.99762809,
       0.99756573, 0.99750337, 0.99744101, 0.99737865, 0.99731629,
       0.99725393, 0.99719157, 0.99712922, 0.99706686, 0.9970045 ,
       0.99686763, 0.99673077, 0.99659391, 0.99645705, 0.99632018,
       0.99618332, 0.99604646, 0.9959096 , 0.99577274, 0.99563587,
       0.99549901, 0.99536215, 0.99522529, 0.99508843, 0.99495156,
       0.9948147 , 0.99467784, 0.99454098, 0.99440411, 0.99426725,
       0.99413039, 0.99399353, 0.99385667, 0.9937198 , 0.99363294,
       0.99354608, 0.99345923, 0.99337237, 0.99328551, 0.99319865,
       0.99311179, 0.99302493, 0.99293807, 0.99285121, 0.99276435,
       0.99267749, 0.99259063, 0.99250377, 0.99241691, 0.99233006,
       0.9922432 , 0.99215634, 0.99206948, 0.99198262, 0.99189576,
       0.9918089 , 0.99172204, 0.99163518, 0.99150729, 0.99137

**COUPON**:
* tenor  --> fisso, lo dividi
* natural --> cambia in ogni sotto periodo

**SWAP**:

It's composed by two leg. The first is fixed and pays a constant swap rate $r^{s}$; the second one is floating and pays a certain coupon related to an index. You get the swap rate by equalizing the values of the two legs.

Forward swap contract: the same as before, but the flows of money starts defered by a certain period.
Both can ask for a swap yield, and be related to a yield term structure.


Basis swap: we have two floating leg. The first one has payment related to a certain index, and the second one to another index plus a spread, which acts like a compensator. Again, you equalize the values of the two legs to get the spread.

# D&C

$$
D=\frac{\sum_{k=1}^{n}\left(t_{k}-t\right) x_{k} B\left(t, t_{k}\right)}{\sum_{k=1}^{n} x_{k} B\left(t, t_{k}\right)}
$$

where:
* $x_k$ = coupons
* $B(t,t_k)$ = discounting bonds
* $t_k-t$

In [10]:
def duration(b,t=None,x=None,T=None,freq=1):
    """
    Compute duration
    
    Args:
        b (ndarray): 1-D array of bond prices
        t (`ndarray`, optional): delta t 
        x (`ndarray`, optional): coupons
        freq (`int`, optional): frequency, needed when t is not specified
        T (`int`, optional): total timeframe, needed when t is not specified
    
    Returns:
        d (`int`): `duration`
    
    """
    if x==None:
        x=interest_from_price(b,freq=freq)
    if t is None:
        t=np.linspace(1,T,len(b))

    n=np.sum(t*b*x)
    d=np.sum(b*x)

    return n/d

$$
C=\frac{\sum_{k=1}^{n}\left(t_{k}-t\right)^{2} x_{k} B\left(t, t_{k}\right)}{\sum_{k=1}^{n} x_{k} B\left(t, t_{k}\right)}
$$

In [11]:
def convexity(b,t=None,x=None,T=None,freq=1):
    """
    Compute duration
    
    Args:
        b (ndarray): 1-D array of bond prices
        t (`ndarray`, optional): delta t 
        x (`ndarray`, optional): coupons
        freq (`int`, optional): frequency, needed when t is not specified
        T (`int`, optional): total timeframe, needed when t is not specified
    
    Returns:
        d (`int`): `duration`
    
    """
    if x==None:
        x=interest_from_price(b,freq=freq)
    if t is None:
        t=np.linspace(1,T,len(b))

    n=np.sum(np.square(t)*b*x)
    d=np.sum(b*x)

    return n/d